In [112]:
import numpy as np
import pandas as pd
import os
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import regex as re
import matplotlib.pyplot as plt
%matplotlib inline

In [84]:
root_dir='/Users/guptadhruv51/Desktop/Github/fake-news'

In [85]:
train_data=pd.read_csv(os.path.join(root_dir+'/train.csv'))
train_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [86]:
test_data=pd.read_csv(os.path.join(root_dir+'/test.csv'))
test_data.head()

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [87]:
X=train_data.drop('label',axis=1)
y=train_data['label']
X.head()

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [88]:
y.value_counts()

1    10413
0    10387
Name: label, dtype: int64

In [89]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer, HashingVectorizer

In [90]:
train_data=train_data.dropna()

In [91]:
messages=train_data.copy()

In [92]:
messages.reset_index(inplace=True)

In [93]:
messages['title'][10]

'Russian Researchers Discover Secret Nazi Military Base ‘Treasure Hunter’ in the Arctic [Photos]'

In [94]:
corpus=[]
ps=PorterStemmer()
for i in range(len(messages)):
    review=re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if word not in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)

In [95]:
#Bag of Words
cv=CountVectorizer(max_features=5000,ngram_range=(1,3))
X=cv.fit_transform(corpus).toarray()

In [96]:
X.shape

(18285, 5000)

In [97]:
y=messages['label']

In [98]:
#Divide Dataset into train and validate

In [99]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.3)

In [100]:
print(X_train.shape,y_train.shape)
print(X_val.shape,y_val.shape)

(12799, 5000) (12799,)
(5486, 5000) (5486,)


In [101]:
corpus_dataset=pd.DataFrame(X_train,columns=cv.get_feature_names())
corpus_dataset.head()

,abandon,abc,abc news,abduct,abe,abedin,abl,abort,abroad,absolut,...,zero,zika,zika viru,zionist,zone,zone new,zone new york,zoo,zu,zuckerberg
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# MultinomialNB Algo

In [115]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import itertools

In [116]:
classifier=MultinomialNB()
classifier.fit(X_train,y_train)
pred=classifier.predict(X_val)
score=metrics.accuracy_score(y_val,pred)
print("Accuracy "+str(score))

Accuracy0.9032081662413416


In [118]:
cm=metrics.confusion_matrix(y_val,pred)
cm

array([[2783,  307],
       [ 224, 2172]])

# Passive Aggressive Algorithm

In [120]:
from sklearn.linear_model import PassiveAggressiveClassifier
clf=PassiveAggressiveClassifier()

In [121]:
clf.fit(X_train,y_train)
pred=clf.predict(X_val)
score=metrics.accuracy_score(y_val,pred)
print("Accuracy "+str(score))
cm=metrics.confusion_matrix(y_val,pred)
cm

Accuracy 0.9174261757200146


array([[2866,  224],
       [ 229, 2167]])

# Multinomial With Hyperparamter

In [122]:
classifier=MultinomialNB(alpha=0.1)

In [123]:
previous_score=0
for alpha in np.arange(0,1,0.1):
    sub_classifier=MultinomialNB(alpha=alpha)
    sub_classifier.fit(X_train,y_train)
    pred=sub_classifier.predict(X_val)
    score=metrics.accuracy_score(y_val,pred)
    if score> previous_score:
        classifier=sub_classifier
    print('Alpha: {}, Score: {}'.format(alpha,score))    

/Users/guptadhruv51/anaconda3/lib/python3.6/site-packages/sklearn/naive_bayes.py:508: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


Alpha: 0.0, Score: 0.8880787458986511
Alpha: 0.1, Score: 0.9019321910317171
Alpha: 0.2, Score: 0.9021144732045207
Alpha: 0.30000000000000004, Score: 0.9033904484141451
Alpha: 0.4, Score: 0.9032081662413416
Alpha: 0.5, Score: 0.9028436018957346
Alpha: 0.6000000000000001, Score: 0.9030258840685381
Alpha: 0.7000000000000001, Score: 0.9030258840685381
Alpha: 0.8, Score: 0.9030258840685381
Alpha: 0.9, Score: 0.9032081662413416


In [126]:
#get Feature Names
feature_names=cv.get_feature_names()

In [132]:
classifier.coef_[0]

array([ -8.79152568,  -8.65618434,  -9.65833636, ..., -10.82840761,
        -8.94809474,  -9.65833636])

In [139]:
#Most Real Words
sorted(zip(classifier.coef_[0],feature_names),reverse=True)[:20]

[(-3.9720625924183226, 'trump'),
 (-4.272734663669251, 'hillari'),
 (-4.356748483787887, 'clinton'),
 (-4.879792575507715, 'elect'),
 (-5.197992642078134, 'new'),
 (-5.230397259442384, 'comment'),
 (-5.268153421860378, 'video'),
 (-5.285402663681375, 'us'),
 (-5.357538479362151, 'hillari clinton'),
 (-5.381191426887991, 'war'),
 (-5.460875175538828, 'email'),
 (-5.466072692436857, 'fbi'),
 (-5.541825500778721, 'vote'),
 (-5.581968505328965, 'world'),
 (-5.611661898254112, 'obama'),
 (-5.7064297296767075, 'russia'),
 (-5.733295012270679, 'america'),
 (-5.753928481557189, 'donald'),
 (-5.782119213779075, 'donald trump'),
 (-5.864010185505369, 'say')]

In [140]:
#Most Fake Words
sorted(zip(classifier.coef_[0],feature_names))[:20]

[(-10.828407611108338, 'abe'),
 (-10.828407611108338, 'abroad'),
 (-10.828407611108338, 'abus new'),
 (-10.828407611108338, 'abus new york'),
 (-10.828407611108338, 'act new'),
 (-10.828407611108338, 'act new york'),
 (-10.828407611108338, 'advic'),
 (-10.828407611108338, 'advis new'),
 (-10.828407611108338, 'advis new york'),
 (-10.828407611108338, 'age new'),
 (-10.828407611108338, 'age new york'),
 (-10.828407611108338, 'agenda breitbart'),
 (-10.828407611108338, 'ahead trump'),
 (-10.828407611108338, 'ail'),
 (-10.828407611108338, 'aleppo new'),
 (-10.828407611108338, 'aleppo new york'),
 (-10.828407611108338, 'ali'),
 (-10.828407611108338, 'america breitbart'),
 (-10.828407611108338, 'america new york'),
 (-10.828407611108338, 'american breitbart')]